In [1]:
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, MetaData, DateTime, Boolean, create_engine
from datetime import datetime


# Metadata ??
metadata = MetaData()

cookies = Table('cookies', metadata, 
    Column('cookie_id', Integer(), primary_key=True),
    Column('cookie_name', String(50), index=True),
    Column('cookie_recipe_url', String(255)),
    Column('cookie_sku', String(55)),
    Column('quantity', Integer()),
    Column('unit_cost', Numeric(12, 2))
)

In [2]:
users = Table('users', metadata, 
    Column('user_id', Integer(), primary_key=True),
    Column('username', String(15), nullable=False, unique=True),
    Column('email_address', String(255), nullable=False),
    Column('phone', String(20), nullable=False),
    Column('password', String(25), nullable=False),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
)

In [3]:
orders = Table('orders', metadata,
    Column('order_id', Integer(), primary_key=True),
    Column('user_id', ForeignKey('users.user_id')),
    Column('shipped', Boolean(), default=False)
)

line_items = Table('line_items', metadata, 
    Column('line_items_id', Integer(), primary_key=True),
    Column('order_id', ForeignKey('orders.order_id')),
    Column('cookie_id', ForeignKey('cookies.cookie_id')),
    Column('quantity', Integer()),
    Column('extended_cost', Numeric(12, 2))
)

# Persisting the Tables

In [4]:
engine = create_engine('sqlite:///cookies.db')
connection = engine.connect()
metadata.create_all(engine)

# Inserting Data

## Single insert

In [5]:
ins = cookies.insert().values(
    cookie_name='chocolate_chip',
    cookie_recipe_url='http://some.aweso.me/cookie/recipe.html',
    cookie_sku='CCO1',
    quantity='12',
    unit_cost='0.60'
)

print(str(ins))

INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)


In [6]:
result = connection.execute(ins)
print(result.inserted_primary_key)

[1]


## Multiple insert

In [7]:
inventory_list = [
    {
        'cookie_name':'peanut_butter',
        'cookie_recipe_url':'https://some.oke.me/cookies/rsia.html',
        'cookie_sku':'PON01',
        'quantity':'24',
        'unit_cost':'0.25'
    },
    {
        'cookie_name': 'oatmeal raisin',
        'cookie_recipe_url': 'http://some.okay.me/cookie/raisin.html',
        'cookie_sku': 'EWW01',
        'quantity': '100',
        'unit_cost': '1.00'
    }
]
result = connection.execute(ins, inventory_list)

# Querying Data

In [8]:
from sqlalchemy import select

s = select([cookies])
rp = connection.execute(s)
result = rp.fetchall()

/home/gagaallall/anaconda3/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:603: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [9]:
result[0].cookie_name

'chocolate_chip'

# Controlling the Columns in the Query

In [10]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
rp = connection.execute(s)
print(rp.keys())
result = rp.first()
print(result)

['cookie_name', 'quantity']
('chocolate_chip', 12)


# Ordering

In [11]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
rp = connection.execute(s)
for cookie in rp:
    print(f'{cookie.quantity} - {cookie.cookie_name}')

12 - chocolate_chip
24 - peanut_butter
100 - oatmeal raisin


## Descending

In [12]:
from sqlalchemy import desc

s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(desc(cookies.c.quantity))
result = connection.execute(s)
print(result.fetchall())
for cookie in result:
    print(cookie)

[('oatmeal raisin', 100), ('peanut_butter', 24), ('chocolate_chip', 12)]


# Limit

In [13]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
s = s.limit(2)
rp = connection.execute(s)
print([result.cookie_name for result in rp])

['chocolate_chip', 'peanut_butter']


# Built-in SQL Functions and Labels

In [14]:
from sqlalchemy import func

s = select([func.sum(cookies.c.quantity)])
rp = connection.execute(s)
print(rp.scalar())

136


In [15]:
s = select([func.count(cookies.c.cookie_name)])
rp = connection.execute(s)
record = rp.first()
print(record.keys())
print(record.count_1)

['count_1']
3


In [16]:
s = select([func.count(cookies.c.cookie_name).label('inventory_count')])
rp = connection.execute(s)
record = rp.first()
print(record.keys())
print(record.inventory_count)

['inventory_count']
3


# Filtering

In [17]:
s = select([cookies]).where(cookies.c.cookie_name == 'chocolate_chip')
rp = connection.execute(s)
record = rp.first()
print(record.items())

[('cookie_id', 1), ('cookie_name', 'chocolate_chip'), ('cookie_recipe_url', 'http://some.aweso.me/cookie/recipe.html'), ('cookie_sku', 'CCO1'), ('quantity', 12), ('unit_cost', Decimal('0.60'))]


In [18]:
s = select([cookies]).where(cookies.c.cookie_name.like('%chocolate%'))
rp = connection.execute(s)
for record in rp.fetchall():
    print(record.cookie_name)

chocolate_chip


# Operaters

In [19]:
s = select([cookies.c.cookie_name, 'SKU-' + cookies.c.cookie_sku])
for row in connection.execute(s):
    print(row)

('chocolate_chip', 'SKU-CCO1')
('peanut_butter', 'SKU-PON01')
('oatmeal raisin', 'SKU-EWW01')


In [20]:
from sqlalchemy import cast

s = select([cookies.c.cookie_name, cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12, 2)).label('inv_cost')])

for row in connection.execute(s):
    print(f'{row.cookie_name} - {row.inv_cost}')

chocolate_chip - 7.20
peanut_butter - 6.00
oatmeal raisin - 100.00
/home/gagaallall/anaconda3/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:603: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [21]:
from sqlalchemy import and_, not_, or_

s = select([cookies]).where(
    and_(
        cookies.c.quantity > 23, 
        cookies.c.unit_cost < 0.40
    )
)

for row in connection.execute(s):
    print(row.cookie_name)

peanut_butter


In [22]:
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    or_(
        cookies.c.quantity.between(10, 50),
        cookies.c.cookie_name.contains('chip')
    )
)
for row in connection.execute(s):
    print(row.cookie_name)


chocolate_chip
peanut_butter


# Updating Data

In [23]:
from sqlalchemy import update

u = update(cookies).where(cookies.c.cookie_name == 'chocolate_chip')
u = u.values(quantity=(cookies.c.quantity + 120))
result = connection.execute(u)
print(result.rowcount)
s = select([cookies]).where(cookies.c.cookie_name == 'chocolate_chip')
result = connection.execute(s).first()
for key in result.keys():
    print(f'{key:>20}: {result[key]}')

1
           cookie_id: 1
         cookie_name: chocolate_chip
   cookie_recipe_url: http://some.aweso.me/cookie/recipe.html
          cookie_sku: CCO1
            quantity: 132
           unit_cost: 0.60


# Deleting Data

In [24]:
from sqlalchemy import delete

d = delete(cookies).where(cookies.c.cookie_name == 'chocolate_chip')
result = connection.execute(d)
print(result.rowcount)

s = select([cookies]).where(cookies.c.cookie_name == 'chocolate_chip')
result = connection.execute(s).fetchall()
print(len(result))

1
0


---------------------------------------------------

In [25]:
customer_list = [
    {
        'username': 'cookiemon',
        'email_address': 'mon@cookie.com',
        'phone': '111-111-1111',
        'password': 'password'
    },
    {
        'username': 'cakeeater',
        'email_address': 'cakeeater@cake.com',
        'phone': '222-222-2222',
        'password': 'password'
    },
    {
        'username': 'pieguy',
        'email_address': 'guy@pie.com',
        'phone': '333-333-3333',
        'password': 'password'
    }
]
ins = users.insert()
result = connection.execute(ins, customer_list)

In [26]:
ins = orders.insert().values(user_id=1, order_id=1)
result = connection.execute(ins)
ins = line_items.insert()
order_items = [
    {
        'order_id': 1,
        'cookie_id': 1,
        'quantity': 2,
        'extended_cost': 1.00
    },
    {
        'order_id': 1,
        'cookie_id': 3,
        'quantity': 12,
        'extended_cost': 3.00
    }
]
result = connection.execute(ins, order_items)
ins = orders.insert().values(user_id=2, order_id=2)
result = connection.execute(ins)
ins = line_items.insert()
order_items = [
    {
        'order_id': 2,
        'cookie_id': 1,
        'quantity': 24,
        'extended_cost': 12.00
    },
    {
        'order_id': 2,
        'cookie_id': 4,
        'quantity': 6,
        'extended_cost': 6.00
    }
]
result = connection.execute(ins, order_items)


NameError: name 'insert' is not defined